# 📊 Код ранжирования изображений по тексту с Florence-2

* принимает **папку `/content/images`** с сохранёнными PNG/JPG,

* читает **`prompts.csv`** или `prompts.json` со столбцами/ключами
  `image, prompt, prompt2, negative, negative2`,

* вычисляет для каждого изображения-строки четыре оценки:

| модель                  | что считает                                  | почему                             |
|------------------------|----------------------------------------------|------------------------------------|
| **SigLIP 2 (ViT-L/14)** | глобальное cos( I, T )                       | надёжная мультиязычная «семантика» |
| **Florence-2**          | число фраз из prompt, найденных на картинке | уточнённое локальное выравнивание  |
| **CLIP-IQA (ViT-B/16)** | вероятность «качественного» фото             | штраф за артефакты                 |
| **DINOv2 (ViT-L/14)**   | визуальный «aesthetic score» (ℓ₂-норма CLS) | доп. сигнал «детальности/качества» |

* собирает ансамбль:
  `S = α · SigLIP – α · SigLIP_neg + β · Florence – β · Florence_neg + γ · IQA + δ · DINO`,

* выдаёт `ranking.csv` с итоговым порядком изображений.

| ячейка | что делает                                                                                 |
|--------|--------------------------------------------------------------------------------------------|
| **0**  | установка зависимостей: PyTorch, Transformers, CLIP-IQA, Florence-2 и др.                   |
| **1**  | загрузка моделей: SigLIP2, Florence-2, DINOv2, CLIP-IQA                                    |
| **2**  | обёртки-функции для получения индивидуальных скорингов                                     |
| **3**  | функция `rank_folder`: ранжирует изображения по агрегированному скору                      |
| **4**  | запуск с указанием папки и prompt-файла, сохранение результата в `ranking.csv`             |

> **Совет по весам:**
> Для стабильности нормализуйте оценки (`z-score` по каждому столбцу) — так проще подбирать α, β, γ, δ.

---

### 🧠 Что считают модели:

* **SigLIP 2** — глобальная семантика между изображением и всей подсказкой;
* **Florence-2** — phrase grounding: доля слов/фраз prompt, для которых найдены регионы на изображении.;
* **CLIP-IQA** — вероятность того, что изображение качественное (vs «bad photo»);
* **DINOv2** — L2-норма CLS-вектора, коррелирующая с визуальной выразительностью.

---

Загрузите картинки в `/content/images` и подготовьте `prompts.csv` вида:

```csv
image,prompt,prompt2,negative,negative2
0001.png,"A serene landscape with mountains","A calm river under pink sunset","low quality","distortions"
0002.png,"A serene landscape with mountains","","low quality",""
```

После выполнения всех ячеек получите `ranking.csv` с отсортированными изображениями.
"""

In [1]:
!pip install --upgrade pip setuptools wheel

In [3]:
%cd rank_images_project
!pip install --pre \
  --extra-index-url https://download.pytorch.org/whl/nightly/cu121 \
  -e .

/content/rank_images_project
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cu121
Obtaining file:///content/rank_images_project
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for rank_images (pyproject.toml) ... done
  Created wheel for rank_images: filename=rank_images-0.1.0-0.editable-py3-none-any.whl size=5813 sha256=2e986b338e27210b628ea47fd6665fca30bec14e9dc2149d35ce4716a69270c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-c_0dahjd/wheels/18/44/46/5787710d98191fa8e596c65b5c1e1e623d554349b5f4ae47df
Successfully built rank_images


In [1]:
%cd rank_images_project
!ls -R

/content/rank_images_project
.:
data  pyproject.toml  README.md  requirements.txt  setup.py  src

./data:
demo_images

./data/demo_images:
1010.png  222.png  444.png  666.png  888.png  prompts.json
111.png   333.png  555.png  777.png  999.png  ranking.csv

./src:
rank_images  rank_images.egg-info

./src/rank_images:
cli.py	   data_processing.py  __init__.py  models.py	 ranking.py
config.py  device_utils.py     metrics.py   __pycache__

./src/rank_images/__pycache__:
cli.cpython-311.pyc		 __init__.cpython-311.pyc
config.cpython-311.pyc		 metrics.cpython-311.pyc
data_processing.cpython-311.pyc  models.cpython-311.pyc
device_utils.cpython-311.pyc	 ranking.cpython-311.pyc

./src/rank_images.egg-info:
dependency_links.txt  PKG-INFO	    SOURCES.txt
entry_points.txt      requires.txt  top_level.txt


После успешной установки вы можете протестировать работу CLI

In [2]:
!rank-images --help

2025-07-23 19:06:08.586050: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-23 19:06:08.604081: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753297568.625368   27150 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753297568.631820   27150 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-23 19:06:08.653585: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
!rank-images --demo

2025-07-23 19:06:24.428486: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-23 19:06:24.445342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753297584.466151   27254 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753297584.472496   27254 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-23 19:06:24.493060: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
!rank-images data/demo_images --prompts data/demo_images/prompts.json

2025-07-23 19:11:56.247531: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-23 19:11:56.264943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753297916.285947   28712 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753297916.292321   28712 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-23 19:11:56.313415: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# настройка логирования

In [1]:
# Первая ячейка в Jupyter-ноутбуке или начало cli.py
import logging
import sys

# --- Настройка логирования ---
# 1. Очищаем существующие обработчики (на случай, если basicConfig уже вызывался)
logging.getLogger().handlers.clear()
logging.getLogger().setLevel(logging.NOTSET)

# 2. Создаем новый обработчик, который будет писать в stdout (вывод ячейки)
handler = logging.StreamHandler(sys.stdout)
# 3. Устанавливаем формат сообщений
formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(name)s: %(message)s")
handler.setFormatter(formatter)
# 4. Добавляем обработчик к корневому логгеру
logging.getLogger().addHandler(handler)
# 5. Устанавливаем УРОВЕНЬ логгирования на INFO
#    Это ключевой момент: только сообщения уровня INFO и выше будут отображаться
logging.getLogger().setLevel(logging.INFO)
# --- Конец настройки логирования ---

In [2]:
%cd rank_images_project
import sys
from pathlib import Path
# Предполагаем, что текущая директория - корень проекта
project_root = Path.cwd()
sys.path.insert(0, str(project_root / "src")) # Добавляем src в путь импорта

from rank_images.models import load_models
from rank_images.ranking import rank_folder

# Загружаем модели один раз
print("Загружаю модели...")
load_models()
print("Модели загружены.")


/content/rank_images_project
2025-07-23 21:24:45,679 [INFO] numexpr.utils: NumExpr defaulting to 12 threads.
Загружаю модели...
2025-07-23 21:24:49,662 [INFO] rank_images.models: Начинаю загрузку моделей (на CPU)...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


2025-07-23 21:24:55,159 [INFO] rank_images.models: Модель CLIP-IQA загружена.
2025-07-23 21:25:04,077 [INFO] rank_images.models: Модель SigLIP-2 загружена.


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


2025-07-23 21:25:06,798 [INFO] rank_images.models: Модель DINOv2 загружена.
2025-07-23 21:25:06,799 [INFO] rank_images.models: Попытка загрузить Florence-2 из локального кэша...


/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


2025-07-23 21:25:07,135 [INFO] accelerate.utils.modeling: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
2025-07-23 21:26:04,411 [INFO] rank_images.models: Florence-2 успешно загружена из локального кэша.
2025-07-23 21:26:07,945 [INFO] rank_images.models: Все модели успешно загружены и готовы к использованию.
Модели загружены.


In [3]:
# Выполняем ранжирование
demo_images_dir = project_root / "data" / "demo_images"
prompts_path = str(demo_images_dir / "prompts.json") # <-- str

print("Начинаю ранжирование...")
result_df = rank_folder(
    img_dir=demo_images_dir,
    #prompts_in="An advertising image for a credit card, featuring prominent 3D rendered numbers representing a high deposit percentage. The scene is set against a stunning natural backdrop of a serene blue ocean meeting majestic, sun-drenched mountains. The overall style should be a vibrant and dynamic 3D render, capturing the feeling of opportunity and natural beauty."
    prompts_in= prompts_path # <-- str
)
print("Ранжирование завершено.")
print(result_df.head())

Начинаю ранжирование...
2025-07-23 21:26:38,641 [INFO] rank_images.ranking: Начинаю ранжирование изображений в папке: /content/rank_images_project/data/demo_images
2025-07-23 21:26:38,642 [INFO] rank_images.data_processing: Загружаю промпты из JSON файла: /content/rank_images_project/data/demo_images/prompts.json
2025-07-23 21:26:38,643 [INFO] rank_images.data_processing: Промпты предоставлены в виде словаря.
2025-07-23 21:26:38,645 [INFO] rank_images.ranking: Начинаю вычисление метрик для изображений...


Обработка изображений:   0%|          | 0/10 [00:00<?, ?it/s]

2025-07-23 21:27:05,139 [INFO] rank_images.ranking: Успешно обработано 10 изображений.
2025-07-23 21:27:05,140 [INFO] rank_images.ranking: Нормализую метрики и вычисляю итоговый балл...
2025-07-23 21:27:05,146 [INFO] rank_images.ranking: Результаты ранжирования сохранены в: /content/rank_images_project/data/demo_images/ranking.csv
Ранжирование завершено.
      image       sig      flor       iqa      dino     total
0   222.png  0.657665  1.159765  0.043162  0.738321  0.723823
1   777.png  1.232570  0.207553 -0.525943  0.175791  0.565349
2   666.png  1.415781  0.207553 -2.087787 -1.300851  0.296036
3  1010.png  0.092237  0.207553  0.323003  1.089902  0.239965
4   888.png -0.113086 -0.730290  1.174593  2.004013  0.057964
